<a href="https://colab.research.google.com/github/yejijang-analyst/ESAA/blob/main/exercise/24-1/Week6_sentimetanalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 아래의 데이터를 이용하여 공부한 내용을 바탕으로 문제 2개를 만들고 답하세요.

In [1]:
import nltk
nltk.download("book", quiet=True)
from nltk.book import *

*** Introductory Examples for the NLTK Book ***
Loading text1, ..., text9 and sent1, ..., sent9
Type the name of the text or sentence to view it.
Type: 'texts()' or 'sents()' to list the materials.
text1: Moby Dick by Herman Melville 1851
text2: Sense and Sensibility by Jane Austen 1811
text3: The Book of Genesis
text4: Inaugural Address Corpus
text5: Chat Corpus
text6: Monty Python and the Holy Grail
text7: Wall Street Journal
text8: Personals Corpus
text9: The Man Who Was Thursday by G . K . Chesterton 1908


In [2]:
# 파일 목록 확인
nltk.corpus.gutenberg.fileids()

['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

In [3]:
hamlet = nltk.corpus.gutenberg.raw('shakespeare-hamlet.txt')
print(hamlet[:10000])

[The Tragedie of Hamlet by William Shakespeare 1599]


Actus Primus. Scoena Prima.

Enter Barnardo and Francisco two Centinels.

  Barnardo. Who's there?
  Fran. Nay answer me: Stand & vnfold
your selfe

   Bar. Long liue the King

   Fran. Barnardo?
  Bar. He

   Fran. You come most carefully vpon your houre

   Bar. 'Tis now strook twelue, get thee to bed Francisco

   Fran. For this releefe much thankes: 'Tis bitter cold,
And I am sicke at heart

   Barn. Haue you had quiet Guard?
  Fran. Not a Mouse stirring

   Barn. Well, goodnight. If you do meet Horatio and
Marcellus, the Riuals of my Watch, bid them make hast.
Enter Horatio and Marcellus.

  Fran. I thinke I heare them. Stand: who's there?
  Hor. Friends to this ground

   Mar. And Leige-men to the Dane

   Fran. Giue you good night

   Mar. O farwel honest Soldier, who hath relieu'd you?
  Fra. Barnardo ha's my place: giue you goodnight.

Exit Fran.

  Mar. Holla Barnardo

   Bar. Say, what is Horatio there?
  Hor. A peece of

### (1) 문제 1 : 문장단위로 분해해봐

In [7]:
import pandas as pd

In [20]:
hamlet_sents = gutenberg.sents('shakespeare-hamlet.txt')

In [21]:
hamlet_sents

[['[', 'The', 'Tragedie', 'of', 'Hamlet', 'by', 'William', 'Shakespeare', '1599', ']'], ['Actus', 'Primus', '.'], ...]

In [22]:
book_sentences = [' '.join(sentence) for sentence in hamlet_sents]
book_df = pd.DataFrame(book_sentences, columns=['sentence'])

In [23]:
book_df

,sentence
0,[ The Tragedie of Hamlet by William Shakespear...
1,Actus Primus .
2,Scoena Prima .
3,Enter Barnardo and Francisco two Centinels .
4,Barnardo .
...,...
3101,"Go , bid the Souldiers shoote ."
3102,Exeunt .
3103,"Marching : after the which , a Peale of Ordena..."
3104,FINIS .


In [24]:
text = book_df.copy()

### (2) 문제 2 감성분석 해봐

In [10]:
from nltk.corpus import wordnet as wn

# 간단한 NTLK PennTreebank Tag를 기반으로 WordNet기반의 품사 Tag로 변환

def penn_to_wn(tag):
    if tag.startswith("J"):
        return wn.ADJ
    elif tag.startswith("N"):
        return wn.NOUN
    elif tag.startswith("R"):
        return wn.ADV
    elif tag.startswith("V"):
        return wn.VERB

In [11]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag

In [13]:
def swn_polarity(text):
    # 감성 지수 초기화
    sentiment = 0.0
    tokens_count = 0

    lemmatizer = WordNetLemmatizer()
    raw_sentences = sent_tokenize(text)

    #분해된 문장별로 단어 토큰 -> 품사 태깅 후에 SentiSynset 생성 -> 감성 지수 합산
    for raw_sentence in raw_sentences:
        #NLTK 기반의 품사 태깅 문장 추출
        tagged_sentence = pos_tag(word_tokenize(raw_sentence))
        for word, tag in tagged_sentence:

            #WordNet 기반 품사 태깅과 어근 추출
            wn_tag = penn_to_wn(tag)
            if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
                continue
            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
            if not lemma:
                continue
            #어근을 추출한 단어와 WordNet 기반 품사 태깅을 입력해 Synset 객체를 생성
            synsets = wn.synsets(lemma, pos=wn_tag)
            if not synsets:
                continue
            #sentiwordnet의 감성 단어 분석으로 감성 synset 추출
            #모든 단어에 대해 긍정 감성 지수는 +로 부정 감성지수는 -로 합산해 감성 지수 계산
            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name())
            sentiment += (swn_synset.pos_score()-swn_synset.neg_score())
            tokens_count += 1

    if not tokens_count:
        return 0

    #총 score가 0 이상일 경우 긍정 1, 그렇지 않을 경우 부정 0 반환
    if sentiment>=0:
        return 1
    return 0

In [15]:
import nltk
nltk.download('sentiwordnet')


[nltk_data] Downloading package sentiwordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/sentiwordnet.zip.


True

In [25]:
book_df["preds"]=book_df["sentence"].apply(lambda x : swn_polarity(x))
preds = book_df["preds"].values

In [26]:
book_df

,sentence,preds
0,[ The Tragedie of Hamlet by William Shakespear...,1
1,Actus Primus .,1
2,Scoena Prima .,1
3,Enter Barnardo and Francisco two Centinels .,0
4,Barnardo .,0
...,...,...
3101,"Go , bid the Souldiers shoote .",1
3102,Exeunt .,0
3103,"Marching : after the which , a Peale of Ordena...",1
3104,FINIS .,1


In [27]:
book_df['preds'].value_counts()

preds
1    1805
0    1301
Name: count, dtype: int64

# 2. 조원들과 만든 문제와 답을 공유해보세요.